In [18]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [19]:
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime

<IPython.core.display.Javascript object>

In [20]:
df = pd.read_csv(r'C:\Users\ventu\Python\Data_Science_Projects\Used Phones\storage.csv',index_col=[0])
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.set_index('Listing')

<class 'pandas.core.frame.DataFrame'>
Index: 33656 entries, 0 to LUFI64736
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Listing      6074 non-null   object
 1   Model        33656 non-null  object
 2   Platform     33656 non-null  object
 3   Carrier      33656 non-null  object
 4   Color        33636 non-null  object
 5   Storage      33656 non-null  object
 6   ListDate     33656 non-null  object
 7   ExpiredDate  19537 non-null  object
 8   SaleDate     9341 non-null   object
 9   Views        33656 non-null  int64 
 10  Quantity     33656 non-null  int64 
 11  Price        33656 non-null  int64 
 12  Condition    33656 non-null  object
 13  Title        32838 non-null  object
 14  Description  33472 non-null  object
 15  Damage       15454 non-null  object
 16  Sold         9341 non-null   object
dtypes: int64(3), object(14)
memory usage: 4.6+ MB


C:\Users\ventu\.conda\envs\data\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<IPython.core.display.Javascript object>

### Preprocessing
1. Transform numeric string values into numeric
2. Encode Categorical Variables
3. Change string dates to dateime

In [21]:
# Removes the GB from storage and turns it into an int (could think about using this as categorical instead)
df.Storage = df.Storage.apply(lambda x: int(re.sub("[^0-9]", "", x)))

# Changing string dates into datetime format
df.ListDate = pd.to_datetime(df.ListDate, format="%b %d, %Y")
df.SaleDate = pd.to_datetime(df.SaleDate, format="%b %d, %Y")
# df.ListDate = df.ListDate.apply(lambda x: datetime.strptime(x, '%b %d, %Y'))
# df.SaleDate = df.SaleDate.apply(lambda x: datetime.strptime(x, '%b %d, %Y') if x != np.NAN else x)


<IPython.core.display.Javascript object>

In [23]:
df.head()

,Listing,Model,Platform,Carrier,Color,Storage,ListDate,ExpiredDate,SaleDate,Views,Quantity,Price,Condition,Title,Description,Damage,Sold
0,LUJS17403,Apple iPhone 11,iOS,AT&T,Black,64,2020-09-08,NaN,"Sep 15, 2020",24,1,645,Mint,NaN,\r\nTested 100% working. Comes with full acces...,NaN,Yes
1,LUDI07758,Apple iPhone 11,iOS,AT&T,Purple,64,2020-03-27,NaN,NaN,389,2,656,Mint,♻️ FREE FAST SHIPPING – AT&T- MINT,This phone is in MINT condition. It is fully ...,NaN,NaN
2,LUJX60358,Apple iPhone 11,iOS,AT&T,Purple,128,2020-09-14,NaN,"Sep 14, 2020",15,1,590,Good,NaN,"\r\nAT&T Apple iPhone 11 128GB UNLOCKED , GREA...","\r\nlight signs of wear, barely seen.\r\n",Yes
3,LUIX98388,Apple iPhone 11 Pro Max,iOS,AT&T,Gray,256,2020-08-19,NaN,NaN,189,1,1079,Mint,NaN,\r\nLike New Device» Mint Condition» Generic B...,NaN,NaN
4,LUJY83673,Apple iPhone 11 Pro Max,iOS,AT&T,Green,64,2020-09-14,"Sep 24, 2020",NaN,21,1,855,Mint,NaN,\r\niPhone 11 Pro Max - green color. Like new ...,NaN,NaN


<IPython.core.display.Javascript object>

0           2020-09-08
1           2020-03-27
2           2020-09-14
3           2020-08-19
4           2020-09-14
               ...    
LUME65204   2020-11-12
LUMG24181   2020-11-14
LUME25584   2020-11-12
LULS15546   2020-10-30
LUFI64736   2020-05-18
Name: ListDate, Length: 33656, dtype: datetime64[ns]

<IPython.core.display.Javascript object>

In [9]:
df["Price"] = pd.to_numeric(df["Price"])
# df.to_csv(r'C:\Users\ventu\pyProjects\Data_Science_Projects\Used Phones\storage.csv')

<IPython.core.display.Javascript object>

In [10]:
df2 = df.loc[df['Sold']=='Yes'].groupby('Model').Price.mean()

<IPython.core.display.Javascript object>

In [10]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
    orientation="h",
    x = df2.values,
    y = df2.index))

fig.update_layout(
    autosize=False,
    height=600,
    yaxis={'categoryorder':'total ascending'},
)

fig.show()

<IPython.core.display.Javascript object>

In [11]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=[20, 14, 23],
            y=['giraffes', 'orangutans', 'monkeys']))

fig.show()

<IPython.core.display.Javascript object>